In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

In [ ]:
df.head()

In [ ]:
df.groupby('type').count()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset = ['cast', 'country', 'rating'])

In [ ]:
movies = df[df['type'] == 'Movie'].reset_index()

In [ ]:
movies.head()

In [ ]:
tv = df[df['type'] == 'TV Show'].reset_index()

In [ ]:
tv.head()

In [ ]:
x= df['type'].value_counts().reset_index()
fig = px.pie(x,values = 'type', names= 'index')
fig.update_traces(textposition = 'inside', textinfo = 'percent+label')

In [ ]:
country_df = df['country'].value_counts().reset_index()
country_df = country_df[country_df['country']/ country_df['country'].sum() > 0.01]

fig = px.pie(country_df, values = 'country', names = 'index', color_discrete_sequence = px.colors.sequential.RdBu)
fig
fig.update_traces(textposition = 'inside', textinfo = 'percent+label')

In [ ]:
net_date = tv[['date_added']].dropna()

In [ ]:
net_date['year'] = net_date['date_added'].apply(lambda x: x.split(', ')[-1])
net_date['month'] = net_date['date_added'].apply(lambda x: x.lstrip().split(' ')[0])

In [ ]:
net_date['month'].unique()

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'][::-1]

In [ ]:
data = net_date.groupby('year')['month'].value_counts().unstack().fillna(0)[month_order].T
plt.figure(figsize = (12,7))
plt.pcolor(data, cmap = 'Blues', edgecolors = 'white', linewidth = 2)
plt.xticks(np.arange(0.5, len(data.columns), 1), data.columns, fontsize=7, fontfamily='serif')
plt.yticks(np.arange(0.5, len(data.index), 1), data.index, fontsize=7, fontfamily='serif')
plt.title('Netflix Contents Update', fontsize=12, fontfamily='calibri', fontweight='bold', position=(0.20, 1.0+0.02))
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize = 8)
cbar.ax.minorticks_on()

In [ ]:
plt.figure(figsize = (12,7))
sns.countplot(x = 'rating', data= movies)

In [ ]:
plt.figure(figsize = (12,7))
sns.countplot(y = 'release_year', data= movies, order = movies['release_year'].value_counts().index[0:15])

In [ ]:
movies['duration'] = movies['duration'].str.replace('min', '')

In [ ]:
movies['duration'] = movies['duration'].astype(str).astype(int)
movies['duration'] 

In [ ]:
sns.kdeplot(data = movies['duration'], shade = True)

In [ ]:
from collections import Counter

genres = list(movies['listed_in'])
gen = []

for i in genres:
    i = list(i.split(','))
    for j in i:
        gen.append(j.replace(' ', ""))
g = Counter(gen)

In [ ]:
from wordcloud import WordCloud, STOPWORDS

text= list(set(gen))
plt.figure(figsize = (13,13))
wordcloud = WordCloud(max_words=1000000,background_color="white").generate(str(text))
plt.imshow(wordcloud,interpolation="bilinear")
plt.show()

In [ ]:
fig = plt.figure(figsize = (12,7))
g = sns.countplot(x = gen, data = movies)
plt.xticks(rotation=90)

In [ ]:
features = ['title', 'duration']
durations = tv[features]

durations['no_of_seasons'] = durations['duration'].str.replace(' Season', '')
durations['no_of_seasons'] = durations['no_of_seasons'].str.replace('s', '')

durations['no_of_seasons']=durations['no_of_seasons'].astype(str).astype(int)

In [ ]:
t = ['title', 'no_of_seasons']
top = durations[t]

top = top.sort_values(by = 'no_of_seasons', ascending = False)

In [ ]:
top20= top[0:20]
top20.plot(kind = 'bar', x='title',y='no_of_seasons', color = 'blue')

In [ ]:
bottom = top.sort_values(by = 'no_of_seasons')
bottom20 = bottom[30:50]
bottom20.plot(kind = 'bar', x='title',y='no_of_seasons', color = 'green')

In [ ]:
genres = list(tv['listed_in'])
gen = []

for i in genres:
    i = list(i.split(','))
    for j in i:
        gen.append(j.replace(' ', ''))
g = Counter(gen)

In [ ]:
text = list(set(gen))
from wordcloud import WordCloud, STOPWORDS
wordcloud = WordCloud(max_words = 10000).generate(str(text))
plt.figure(figsize = (13,13))
plt.imshow(wordcloud,interpolation="bilinear")

In [ ]:
us_series = tv[tv['country'] == 'United States']

In [ ]:
oldest_us_series = us_series.sort_values(by = 'release_year')[0:20]
oldest_us_series.plot(kind = 'bar', x = 'title', y='release_year')

In [ ]:
newest_us_series = us_series.sort_values(by = 'release_year', ascending = False)[0:20]
newest_us_series.plot(kind = 'bar', x = 'title', y='release_year')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words = 'english')

df['description'] = df['description'].fillna('')

tfidf_matrix = tfidf.fit_transform(df['description'])
tfidf_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index, index = df['title']).drop_duplicates()

In [ ]:
indices

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Peaky Blinders')

In [ ]:
get_recommendations('Friends')